This notebook imports the fundamental objects of the streamm.forcefields module and goes through the functionality of each

In [1]:
import streamm.forcefields.particletype as particletype

Let's start with a methane molecule from the structure.ipynb example 

We have an sp3 carbon bonded to hydrogens

Let's create the force field parameters object for a 'CT' carbon and a HC hydrogen

In [2]:
CT = particletype.Particletype('CT')
HC = particletype.Particletype('HC')

Set some parameters from J. Am. Chem. Soc., Vol. 121, No. 20, 1999

In general you should pick a force field that has been shown to work well for your system and set up the parameters 

In [3]:
HC.epsilon = 0.03 # kcal/mol
HC.sigma = 2.5 # Angstroms 

In [4]:
CT.epsilon = 0.066 # kcal/mol
CT.sigma = 3.5 # Angstroms 

Set the bond stretching parameters 

In [5]:
import streamm.forcefields.bondtype as bondtype

In [6]:
C_H = bondtype.Bondtype('CT','HC')

In [7]:
C_H.setharmonic(1.080,367.0)

In [8]:
print C_H

 bond  CT - HC type harmonic 
  harmonic r_0 = 1.080000 K = 367.000000 lammps index 0  gromacs index 0  


In [9]:
import streamm.forcefields.angletype as angletype

In [10]:
H_C_H = angletype.Angletype('HC','CT','HC')

In [11]:
H_C_H.setharmonic(110.7,37.50)

In [12]:
print H_C_H

 angle  HC - CT - HC type harmonic 
  harmonic theta_0 = 110.700000 K = 37.500000 lammps index 0  gromacs index 0  


Let's create a parameter container to keep track of our parameters

In [13]:
import streamm.forcefields.container as container 

In [14]:
paramC = container.Container('oplsaa')

Add parameters to the container

In [15]:
paramC.add_particletype(CT)

In [16]:
paramC.add_particletype(HC)

In [17]:
paramC.add_bondtype(C_H)

In [18]:
paramC.add_angletype(H_C_H)

In [19]:
print paramC


    Parameters 
      LJ parameters 2 
      Bond parameters 1 
      Angle parameters 1 
      Dihedral parameters 0 
      Imporper Dihedral parameters 0 



In [20]:
print paramC.tag

oplsaa


In [21]:
print paramC.dump_pickle()

None


Read in methane .xyz file from the structures example

In [22]:
import streamm.buildingblocks.container as bb

In [ ]:
me = bb.Container('methane')

In [ ]:
me.read_xyz()

Set the ffkey to corispond to particletype 

In [ ]:

for pkey,p in me.particles.iteritems():
    if( p.symbol == 'C' ):
        p.ffkey = 'CT'
    elif( p.symbol == 'H' ):
        p.ffkey = 'HA'

In [ ]:
mol_i.bonded_nblist = mol_i.guess_nblist(0,radii_buffer=1.25)

Find bonds and bond angles based on neighbor list 

In [ ]:
mol_i.bonded_bonds()
mol_i.bonded_angles()

In [ ]:
print me.n_particles

Now we can set the particles, bonds and bond angles of the molecule to have parameters associated with the ffkey

In [ ]:
C_C = bondtype.Bondtype('CT','CT')

In [ ]:
C_C.setharmonic(1.529,268.0)

In [ ]:
print C_C

In [ ]:
import math

Create a particle object with tag methane

In [ ]:
mol_i = bb.Container('methane')

In [ ]:
print(mol_i)

You can read in the .xyz file from the structures example or create a methane geometry using a molecular viewer such as Avogadro (https://avogadro.cc/)

In [ ]:
mol_i.read_xyz()

In [ ]:
mol_i.bonded_nblist = mol_i.guess_nblist(0,radii_buffer=1.25)

Check that all the particles have been read in

In [ ]:
print mol_i.n_particles

Check that the neighbor list was set correctly 

In [ ]:
print mol_i.bonded_nblist

Looks good, you should have the geometry of a methane molecule with a C-H bond length of 1.2 Angstroms 

We want to use the functionality of the buildingblock object to join methane's together to create alkyl chains of any length

So let's set two of the hydrogens to be reactive sites (rsites). 

You can view the numerical order of the atoms in Avogadro by setting the label to "atom number," however, avogadro labels for 1-N, while stream uses 0 - N-1

We will choose the first two hydrogens and set their rsite variable to 'RH'. It does not matter what this identifier is, as long as the same identifier is passed to the attach() function later. Also, if the identifiers are not unique the order in which it appears in the particles list will also be used.  

In [ ]:
mol_i.particles[1].rsite = 'RH'

In [ ]:
mol_i.particles[2].rsite = 'RH'

Now use the find_rsites() function to create the dictionary of lists to be used by the attach() function

In [ ]:
mol_i.find_rsites()

In [ ]:
print mol_i.show_rsites()

Pass the molecule to the attach function and set the rsite id's and the list positions of the rsites

In [ ]:
mol_j = bb.attach(mol_i,mol_i,'RH',0,'RH',1,tag='ethane')

Write the .xyz to file to be viewed with a molecular viewer. 

In [ ]:
mol_j.write_xyz()

While the ethane molecule was generated, the hydrogens are eclipsed rather than staggered. 

We can avoid this by using the prepattach() function to orient the molecule and remove the reactive site

In [ ]:
mol_k = mol_i.prepattach('RH',0,dir=-1,yangle=angle_rad)

Then apply a shift to set the bond length

In [ ]:
CC_bl = mol_i.particles[0].bonded_radius*2.0
mol_k.shift_pos([CC_bl,0.0,0.0])

Then apply a rotation to set the conformermation to staggered. Use a 180.0 degree rotation to place the reactive site in the correct orientation for subsequent attachments.  

In [ ]:
angle_rad = 180.0*math.pi/180.0 
mol_k.rotate_yz(angle_rad)

In [ ]:
mol_l = mol_i.prepattach('RH',1,dir=1)

In [ ]:
mol_m = bb.attachprep(mol_k,mol_l)

In [ ]:
mol_m.write_xyz('t2.xyz')

In [ ]:
print mol_m.show_rsites()

Attachements can also be done in a loop 

In [ ]:
alkly_n = (12-1)/2 # Number of ethanes to add to get a dodecyl 

In [ ]:
print alkly_n

In [ ]:
mol_n = mol_m 

In [ ]:
mol_n.find_rsites()

In [ ]:
print mol_n.show_rsites()

In [ ]:
for i in range(alkly_n):
    mol_n = bb.attach(mol_n,mol_m,'RH',1,'RH',0)

In [ ]:
mol_n.tag = 'dodecyl'

In [ ]:
mol_n.write_xyz()

Looks good.